# Plot EDGAR dataset
The dataset is from timestamp **2016-10-01 00:00:00** to **2016-12-31 23:00:00**  
**requests** represents the maximum number of requests received by the webserver within the given hour, with a resolution of 1s

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
def parse_datetime(x):
    # return pandas datetime
    return pd.to_datetime(x, format='%Y-%m-%d %H:%M:%S')

series = pd.read_csv(
    './processed/edgar.csv', # filename
    header=None, # contains no header
    index_col=0, # set datetime column as index
    names=['datetime', 'requests'], # name the columns
    converters={'datetime': parse_datetime}, # custom datetime parser
    squeeze=True, # convert to Series
    dtype={'requests': np.float64} # https://git.io/vdbyk
)

In [ ]:
print('--- HEAD ---')
print(series.head())
print('--- TAIL ---')
print(series.tail())

series.plot();

# Naive forecast
The last observed value will occur in the next time interval

In [ ]:
print(series[series.last_valid_index()])

# Autoregressive (AR)
It is a regression of the variable against itself.  
The model is represented as *AR(p)*, where the *AR* process depends on *p* past observations.  

We will be computing *AR(1)*, which is equivalent to *ARIMA(1,0,0)* and *ARMA(1,0)*.

In [ ]:
from statsmodels.tsa.arima_model import ARMA

ar = ARMA(series, order=(1,0)) # (p,q) = (1,0)
ar_fit = ar.fit()

print(ar_fit.forecast(48)[0])
ar_fit.plot_predict(start=series.tail(100).index[0],
                    end=series.last_valid_index() + pd.Timedelta(hours=48),
                    alpha=0.90);

# Autoregressive Moving Averages (ARMA)
The model is represented as *ARMA(p,q)*,  
where the *AR* process depends on *p* past observations and *MA* process depends on *q* past observations.  

We will be computing *ARMA(1,1)*, which is equivalent to *ARIMA(1,0,1)*.

In [ ]:
from statsmodels.tsa.arima_model import ARMA

arma = ARMA(series, order=(1,1)) # (p,q) = (1,1)
arma_fit = arma.fit()

print(arma_fit.forecast(48)[0])
arma_fit.plot_predict(start=series.tail(100).index[0],
                      end=series.last_valid_index() + pd.Timedelta(hours=48),
                      alpha=0.90);

# Autoregressive Integrated Moving Averages (ARIMA)
The model is represented as *ARIMA(p,d,q)*,  
where the *AR* process depends on *p* past observations, *MA* process depends on *q* past observations,  
and *d* represents the order of integration (to make time-series stationary).  

We will be computing R's equivalent of *auto.ARIMA()*.

### Check for stationarity
- **p-value** should be < 0.05
- **Test Statistic** should be < **Critical Value (5%)**

[Read More](https://datascience.ibm.com/exchange/public/entry/view/815137c868b916821dec777bdc23013c)

In [ ]:
from statsmodels.tsa.stattools import adfuller

# Rolling statistics
rolmean = series.rolling(window=52, center=False).mean()
rolstd = series.rolling(window=52, center=False).std()

orig = plt.plot(series, color='blue', label='Original')
mean = plt.plot(rolmean, color='red', label='Rolling Mean')
std = plt.plot(rolstd, color='black', label = 'Rolling Std')
plt.legend(loc='best')
plt.title('Rolling Mean & Standard Deviation')
plt.show(block=False)

# Dickey-Fuller test
dftest = adfuller(series, autolag='AIC')
dfoutput = pd.Series(dftest[0:4],
                     index=['Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)' % key] = value
print(dfoutput)

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import arma_order_select_ic
import warnings

# Select p and q
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    arma_params = arma_order_select_ic(series, fit_kw=dict(method='css'))

p = arma_params.bic_min_order[0]
q = arma_params.bic_min_order[1]

arima = ARIMA(series, order=(p,0,q))
arima_fit = arima.fit()

print(arima_fit.forecast(48)[0])
arima_fit.plot_predict(start=series.tail(100).index[0],
                       end=series.last_valid_index() + pd.Timedelta(hours=48),
                       alpha=0.90);

# Forecasting using R

## Installing forecast

In [ ]:
import rpy2.robjects.packages as rpackages
import warnings

# select mirror
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1)

with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    utils.install_packages('forecast')

## Setup forecast

In [ ]:
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

forecast = rpackages.importr('forecast')
ts = robjects.r('ts')

pandas2ri.activate()
rdata = ts(series.values)

## AR (1)

In [ ]:
ar_fit = forecast.Arima(rdata, robjects.FloatVector((1,0,0)))
ar_forecast = forecast.forecast(ar_fit, h=24)
print(np.asarray(ar_forecast[3]))

## ARMA (1,1)

In [ ]:
arma_fit = forecast.Arima(rdata, robjects.FloatVector((1,0,1)))
arma_forecast = forecast.forecast(arma_fit, h=24)
print(np.asarray(arma_forecast[3]))

## ARIMA

In [ ]:
arima_fit = forecast.auto_arima(rdata)
arima_forecast = forecast.forecast(arima_fit, h=24)
print(np.asarray(arima_forecast[3]))

## ETS

In [ ]:
ets_fit = forecast.ets(rdata)
ets_forecast = forecast.forecast(ets_fit, h=24)
print(np.asarray(ets_forecast[1]))